# Multi-Omics Analysis for LUSC Survival Prediction

Bla



## Imports and helper functions

In [1]:
import os,shutil,io,json,glob,tarfile,requests
import functools

import numpy as np
import pandas as pd

from pypgatk.cgenomes.cbioportal_downloader import CbioPortalDownloadService

In [2]:
def untar_study(study, output_directory, fmt=".tar.gz"):
    file = tarfile.open(
        os.path.join(
            output_directory,
            "{study}{format}".format(study=study, format=fmt)
            )
        )
    file.extractall(output_directory)
    file.close()

In [3]:
def download_cbioportal_study(
        config_file,
        study,
        output_directory = "./data",
        list_studies = False,
        multithreading = True,
        **kwargs
        ):
    pipeline_arguments = {
        CbioPortalDownloadService.CONFIG_OUTPUT_DIRECTORY: output_directory,
        CbioPortalDownloadService.CONFIG_LIST_STUDIES: list_studies,
        CbioPortalDownloadService.CONFIG_MULTITHREADING: multithreading,
        **kwargs,
    } 
    cbioportal_downloader_service = CbioPortalDownloadService(config_file, pipeline_arguments)
    cbioportal_downloader_service.download_study(study)

In [4]:
def get_mirna_files(
        project_id="TCGA-LUSC",
        maxfiles=10000
        ):

    cases_endpt = "https://api.gdc.cancer.gov/files"
    data_endpt = "https://api.gdc.cancer.gov/data"

    # Retrieve associated file names
    filters = {
        "op": "and",
        "content":[
            {"op": "=",
            "content":{
                "field": "cases.project.project_id",
                "value": ["TCGA-LUSC"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.experimental_strategy",
                "value": ["miRNA-Seq"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.data_category",
                "value": ["Transcriptome Profiling"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.data_type",
                "value": ["miRNA Expression Quantification"]
                }
            }
        ]
    }

    params = {
        "filters": json.dumps(filters),
        "fields": ",".join(["cases.submitter_id","file_name"]),
        "format": "TSV",
        "size": str(maxfiles)
    }

    response = requests.get(cases_endpt, params = params)
    files_df = pd.read_csv(io.StringIO(response.text), sep="\t")
    return files_df

def download_mirna_files(
        files_df,
        output_directory = "./data",
        project_id="TCGA-LUSC",
        maxfiles=10000
        ):
    
    params = {"ids": files_df["id"].tolist()}

    response = requests.post(data_endpt,
                            data = json.dumps(params),
                            headers={
                                "Content-Type": "application/json"
                                })

    response_head_cd = response.headers["Content-Disposition"]
    file_name = re.findall("filename=(.+)", response_head_cd)[0]
    with open(os.path.join(output_directory, file_name), "wb") as output_file:
        output_file.write(response.content)
    return file_name

In [5]:
def untar_and_merge_mirna_files(
        files_df,
        file_name,
        output_directory = "./data",
        cleanup=True
        ):
    untar_study(file_name, output_directory, fmt="")
    
    miRNA_IDs = set()
    patient_dfs = {}
    patient_folders = []
    for i in range(len(files_df)):
        patient_id = files_df["cases.0.submitter_id"].iloc[i]
        foldername = files_df["id"].iloc[i]
        patient_fname = os.listdir(os.path.join(output_directory, foldername))[0]
        patient_df = pd.read_csv(os.path.join(output_directory, foldername, patient_fname), sep="\t")
        
        miRNA_IDs.update(patient_df["miRNA_ID"].tolist())
        patient_dfs[patient_id] = patient_df
        patient_folders.append(foldername)
    
    miRNA_df = pd.DataFrame({"patient_id":[], **{k:[] for k in miRNA_IDs}}).set_index("patient_id")
    for patient_id in patient_dfs:
        patient_df = patient_dfs[patient_id]
        patient_id = "{}-01".format(patient_id) # This line is to match cBioPortal's format
        transposed_patient_df = patient_df[["miRNA_ID","reads_per_million_miRNA_mapped"]].set_index("miRNA_ID").transpose()
        transposed_patient_df["patient_id"] = [patient_id]
        transposed_patient_df = transposed_patient_df.set_index("patient_id")
        miRNA_df.loc[patient_id,miRNA_df.columns] = transposed_patient_df[miRNA_df.columns].values.flatten()
        
    if cleanup:
        for patient_folder in patient_folders:
            shutil.rmtree(os.path.join(output_directory, patient_folder))
    
    return miRNA_df

In [6]:
def remove_constant_columns(df):
    columns_to_remove = []
    for idx, column in enumerate(df.columns):
        try:
            if (df[column].std() == 0).any():
                columns_to_remove.append(column)
        except KeyError:
            columns_to_remove.append(column)
    return df.drop(columns=columns_to_remove)

## Data Acquisition and Pre-Processing

### Acquiring data from cBioPortal

In [7]:
cbioportal_config = "./config/cbioportal_config.yaml"
data_directory = "./data"
study_name = "lusc_tcga"

In [8]:
download_cbioportal_study(cbioportal_config, study_name, data_directory)

DEBUG:pypgatk.toolbox.general:The following study 'lusc_tcga' has been downloaded. 


In [9]:
untar_study(study_name, data_directory, fmt=".tar")

In [10]:
RNAseq = pd.read_csv(os.path.join(data_directory, study_name, "data_RNA_Seq_v2_expression_median.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")
linearCNA = pd.read_csv(os.path.join(data_directory, study_name, "data_linear_CNA.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")
methylation_hm450 = pd.read_csv(os.path.join(data_directory, study_name, "data_methylation_hm450.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")

len(RNAseq), len(linearCNA), len(methylation_hm450)

(20531, 24776, 16714)

Remove any constant columns

In [11]:
RNAseq_t = remove_constant_columns(RNAseq.transpose())
RNAseq = RNAseq_t.transpose()
linearCNA_t = remove_constant_columns(linearCNA.transpose())
linearCNA = linearCNA_t.transpose()
methylation_hm450_t = remove_constant_columns(methylation_hm450.transpose())
methylation_hm450 = methylation_hm450_t.transpose()

len(RNAseq), len(linearCNA), len(methylation_hm450)

(20242, 24776, 16714)

### miRNA data from gdc.cancer.gov

In [12]:
mirna_files_df = get_mirna_files()
mirna_files_df.describe()

,cases.0.submitter_id,file_name,id
count,523,523,523
unique,478,523,523
top,TCGA-43-6771,7df584d3-5b11-4987-8227-6d809e8305af.mirbase21...,07fa22d0-99b1-4364-9b93-d937005a2416
freq,2,1,1


In [13]:
#mirna_fname = download_mirna_files(mirna_files_df, data_directory)
mirna_fname = "gdc_download_20211114_175949.738652.tar.gz"

In [14]:
miRNA_df_t = untar_and_merge_mirna_files(mirna_files_df, mirna_fname, data_directory)
print("All columns:", len(miRNA_df_t.columns))
miRNA_df_t = remove_constant_columns(miRNA_df_t)
print("Only non-constant columns:", len(miRNA_df_t.columns))
miRNA_df = miRNA_df_t.transpose()

All columns: 1881
Only non-constant columns: 1579


## Reconciling data from cBioPortal and gdc.cancer.gov

In [15]:
RNAseq_patients = set(RNAseq_t.index)
linearCNA_patients = set(linearCNA_t.index)
methylation_hm450_patients = set(methylation_hm450_t.index)
miRNA_patients = set(miRNA_df_t.index)
all_sets = [RNAseq_patients, linearCNA_patients, methylation_hm450_patients, miRNA_patients]
all_patients = functools.reduce(lambda x, y: x|y, all_sets, set())
patients_on_all_datasets = functools.reduce(lambda x, y: x&y, all_sets, all_patients)

In [17]:
list(map(len, all_sets))

[501, 501, 370, 478]

In [16]:
len(all_patients), len(patients_on_all_datasets)

(504, 362)